# DeepSORT - Lab 7.5

## Recap
This is the Lab using a CNN based algorithm, DeepSORT to track multiple people as part of CE6003's Object Tracking section. You should complete the tasks in this lab as part of the DeepSORT section of the lesson.

Please remember this lab must be completed before taking the quiz at the end of this lesson.

First, if we haven't already done so, we need to clone the various images and resources needed to run these labs into our workspace.

# Introduction

This is an implementation, by Z. Q. Pei of a multiple object tracking algorithm, namely DeepSORT. DeepSORT is basically the same as SORT but with an added CNN model to extract features in image of human part bounded by a detector. 

This CNN model is a RE-ID model and the detector used in the paper documenting it, by Wokje et al, is FasterRCNN, and the original source code for DeepSORT is linked below.

In the original code, the CNN model was implemented in tensorflow, but Z. Q. Pei re-implemented the CNN feature extraction model with PyTorch and this is the code we're working with today. 

Also, Z. Q. Pei used YOLOv3 to generate bboxes instead of FasterRCNN.

It's an interesting relatively recent approach, leading to the intuition that we can effectively have a relatively low cost tracker which uses feature vectors extracted from later layers of detectors as the basis of a cost-function to either enhance or replace a traditional Kalman Filter based tracking approach.

See the video accompanying this lab for a review of recent CNN-based tracking approaches.

# Images and Code

First, check out the DeepSORT algorithm etc and a video to run the demonstration.

In [1]:
import os

os.environ['http_proxy'] = 'http://proxy-chain.intel.com:911'
os.environ['HTTP_PROXY'] = 'http://proxy-chain.intel.com:911'
os.environ['https_proxy'] = 'https://proxy-chain.intel.com:912'
os.environ['HTTPS_PROXY'] = 'https://proxy-chain.intel.com:912'

In [2]:
#!git clone https://github.com/EmdaloTechnologies/CE6003.git
#!git clone https://github.com/mcnamarad1971/CE6003.git

# git clone a pytorch re-implementation of Wolke et al's SORT by ZQ Pei
!git clone https://github.com/ZQPei/deep_sort_pytorch.git


Cloning into 'deep_sort_pytorch'...


**Install YOLO3 Weights**

Download a set of YOLOv3 weights curated by Joseph Redmon.  DeepSORT will use YOLOv3 to do its detections.

In [4]:
#!wget https://pjreddie.com/media/files/yolov3.weights
#!cp /content/yolov3.weights /content/deep_sort_pytorch/YOLOv3
os.system("wget https://pjreddie.com/media/files/yolov3.weights")
os.system("cp /content/yolov3.weights /content/deep_sort_pytorch/YOLOv3")

1

**DeepSort**

Download DeepSORT Checkpoint ckpt.t7, curated by ZQ Pei. DeepSORT will use this Checkpoint to measure similarity between the objects in bounding boxes.



In [6]:
# download cached copy of ZQ Pei's ckpt.t7 checkpoint for deepsort
#!cp /content/CE6003/code/ckpt.t7 /content/deep_sort_pytorch/deep_sort/deep/checkpoint
os.system("cp /content/CE6003/code/ckpt.t7 /content/deep_sort_pytorch/deep_sort/deep/checkpoint")

1

Now, let's run the default demo over our ```pedestrians.mp4``` video.  Now that we have supplied weights for Yolo and parameters for Deep SORT the defaults are sufficient for us today.

```demo_yolo3_deepsort.py VIDEO_PATH```

$\qquad$ ```[--help] ```

$\qquad$ ```[--yolo_cfg YOLO_CFG]```

$\qquad$ ```[--yolo_weights YOLO_WEIGHTS]```

$\qquad$ ```[--yolo_names YOLO_NAMES]```

$\qquad$ ```[--conf_thresh CONF_THRESH]```

$\qquad$ ```[--nms_thresh NMS_THRESH]```

$\qquad$ ```[--deepsort_checkpoint DEEPSORT_CHECKPOINT]```

$\qquad$ ```[--max_dist MAX_DIST]```

$\qquad$ ```[--ignore_display]```

$\qquad$ ```[--display_width DISPLAY_WIDTH]```

$\qquad$ ```[--display_height DISPLAY_HEIGHT]```

$\qquad$ ```[--save_path SAVE_PATH]```

$\qquad$ ```[--use_cuda USE_CUDA]```


In [7]:
#!cd /content/deep_sort_pytorch && python demo_yolo3_deepsort.py --use_cuda=False --ignore_display --save_path=/content/video.mp4 /content/CE6003/images/lab7/vids/pedestrians.mp4
os.system("cd /content/deep_sort_pytorch && python demo_yolo3_deepsort.py --use_cuda=False --ignore_display --save_path=/content/video.mp4 /content/CE6003/images/lab7/vids/pedestrians.mp4")

1

**Video**

Thia code plays the video we just created.

As you can see, adding the concept of tracking features in a more human-like manner, by adding significant weight to the intuition that two detections - one in previous frame and one in current frame - look similar to each other significantly boosts a tracker's performance.

In [10]:
import cv2
import base64
import time
from IPython.display import clear_output, Image, display
#from google.colab.patches import cv2_imshow

def arrayShow (imageArray):
    ret, png = cv2.imencode('.png', imageArray)
    encoded = base64.b64encode(png)
    return Image(data=encoded.decode('ascii'))

video = cv2.VideoCapture("video.mp4")
while(video.isOpened()):
        clear_output(wait=True)
        ret, frame = video.read()
        if(ret == False):
          break
        lines, columns, _ =  frame.shape
        img = arrayShow(frame)
        display(img)
        time.sleep(1)


# Conclusion

## Exercises
**Project 1**
This may form the basis of a reasonably sized project.  DeepSORT uses a second CNN to determine association.  It may be interesting to review the original DeepSORT / SORT papers etc and determine if an approach that sends feature descriptors from late layers in detectors such as YOLOv3, SSD, FasterRCNN along with bounding boxes combined with a Hungarian/Kalman has been thoroughly explored already or if there's scope for novel work.

This is not a requirement for completing this lab!


## Takeaways
1. You've seen evidence that tracking objects based on what they look like instead of based on models of their motion etc. are very powerful.
2. You've seen an example of such an algorithm in play.
3. You've an intuition that there seems to be a lower computational cost approach possible - where features are delivered with bounding boxes for assignment via auction cost algorithm.
4. You might be interested in undertaking a project to determine how effective that approach is.

## References
* ZQ Pei pytorch variant of Deep SORT https://github.com/ZQPei/deep_sort_pytorch
* Wojke et al, DeepSORT paper, https://arxiv.org/pdf/1703.07402.pdf
* Wojke et al, DeepSORT code, https://github.com/nwojke/deep_sort
* Redmon et al, YOLO3 paper, https://pjreddie.com/media/files/papers/YOLOv3.pdf
* Redmon et al, YOLOv3 code, https://pjreddie.com/darknet/yolo/
* Bewley et al, SORT code, https://github.com/abewley/sort
